In [ ]:
kubectl create configmap main-sh --from-file=k8s-api-client.sh

kubectl create configmap dcobj-sample --from-file=dcobjs.json

@"
apiVersion: v1
kind: Pod
metadata:
  name: middleware-service
  labels:
    environment: production
    app: nginx

# service account to read ns and pods
spec:
  serviceAccountName: cp-api-explorer

# folder served by web server
  volumes:
  - name: shared-data
    emptyDir: {}
  # sample file
  - name: dcobj-sample
    configMap:
      name: dcobj-sample
  # middleware script for worker
  - name: main-sh
    configMap:
        name: main-sh

  containers:

  - name: web
    image: nginx
    volumeMounts:
    
    - name: shared-data
      mountPath: /usr/share/nginx/html

  - name: worker
    image: ubuntu
    volumeMounts:
    - name: shared-data
      mountPath: /pod-data
    - mountPath: /usr/local/main
      name: main-sh
      readOnly: true
    command: ["/bin/bash"]
    args:
      - "-c"
      - "date | tee /pod-data/index.html; . /usr/local/main/k8s-api-client.sh"
"@ | kubectl apply -f -

In [ ]:
kubectl describe pod middleware-service

In [ ]:
kubectl logs  middleware-service -c worker

# kubectl logs  middleware-service -c worker -f


In [ ]:
# expose web server on node port
kubectl expose pod middleware-service --type=NodePort --port=80

# minikube service --url middleware-service 

In [ ]:
# create new NS
kubectl create namespace lab

# start some services
1..5 | % {
    $index = $_
    kubectl run "web-$index" --image nginx -n lab
}

In [ ]:
kubectl get pods -A -o wide

In [ ]:
# remove lab pods
kubectl delete namespace lab

In [ ]:
# clean up
kubectl delete pod middleware-service

kubectl delete namespace lab

In [ ]:
$addr = minikube service --url middleware-service | cut -d "/" -f 3
$addr

In [ ]:
ngrok http $addr